삭제 시, 실제 삭제가 아닌, 삭제 표시만 한 뒤에 더미라고 표시하는데,
왜 이를 상수나 어떤 수가 아닌 객체로 표현하는가? 
숫자나 "dummy"라는 문자열인 경우 <=> 비교 비용이 많이 드나? 혹은 데이터가 겹칠 수 있는 문제라던가
객체를 사용하면 참조하는 주소만 갖고 있으면 되나? 어떤 변수가 어떤 객체인지 비교하는 연산은 어떤식으로 진행되나
Dummy객체를 만든다는 것은 진짜 유일한 더미라는 상태를 정의하는 것과 같다. 엄청 큰 숫자나, 잘 사용되지 않는 단어를 더미로 정의할 수 있겠지만 발생가능한 미지의 예외를 사전에 방지하는 것일 수 있겠다. (메모리를 소비해가며)


맴버 변수 중에서 특히 리스트를 결정해야하는데

방법 1. 이차원 배열 사용 => key와 value를 한 곳에 저장 
    예시 
    list  = [    [key1, value1],    [key2, value2],     [key3, value3] , ...    ]   


    장점 : key와 value가 언제나 같은 곳에 저장됨
    단점 : key을 찾기위해서 list안에 또 들어가야 하는 문제가 생김   
          ##검증필요## List는 List[1][1]을 참조, List[1][1]는 key1을 참조?    -> 이중 배열은 더블포인터가 아닌 거 같은데 작동하는 게..

방법 2. 동일한 크기의 리스트를 두 개(이상) 사용 
    예:  keyList와 ValueList 사용        
        keyList = [ key1, key2, ...]
        valueList = [ value1, value2, ...]

    장점 : 인덱스를 통해 바로 해당 값에 접근 가능하니까 좋다
    단점 : 하지만 언제나 동일한 인덱스 상에 key와 value가 존재해서 서로 어긋나지 않도록 관리에 주의


나는 방법 2 선택

<pre>
<h1>
HashTable class
</h1>

                                key  => f(key)  => [ ]

어떤 값을 그냥 순서대로 저장한다면, 찾거나, 삭제하고 수정하기 위해 다시 순서를 순회해야한다. 
저장할 장소릴 미리 정해두면 어떨까. 예를 들어 다이소에 간다면, 물건을 찾기위해 다이소 전체를 훑는 게 아니라(이유없이 그냥 돌아다니면 너무 재밌다)
상품이 속한 카테고리가 적힌 곳을 찾는다면, 그곳에서 원하는 걸 쉽게 발견할 수 있다. 
이처럼 데이터도 생긴 모양에 따라서 저장하는 규칙에 맞게 저장하면 효율적인 탐색을 할 수 있다. 이를 통해 빠른 수정과 삭제도 가능해진다.

칫솔과 비누는 목욕용품 코너에, 종이와 연필은 사무용품 코너이 정리하듯, 
물건과 선반의 짝을 지어주는 규칙을 Hash function이라고 한다. 그리고 이를 통해 저장하는 자료구조를 Hash Table이라고 하며, 데이터가 들어올 수 있는 테이블 내의 각 공간을 버킷이라고 한다.

생각할 수 있는 문제 중 하나는 바로, 이미 짝이 있는 경우다. (난 왜 짝이 없는걸까)
이런 경우, 빈 자리를 찾아서 가는 방법을 생각할 수 있다. 꽉 찬 곳 바로 다음 빈자리를 찾아갈 수도 있고, 한 칸 혹은 여러 칸씩 건너 띄어서 자리를 찾아 떠날 수 있다. 이런 방법을 충돌 회피 기법이라고 한다.

다만 어떤 방법으로 중복을 피하는가에 따라서, 연속적으로 자료가 꽉 찬 덩어리가 생길 수 있는데, 이를 Cluster라고 하며, 이 클러스터가 커질 수록 해시 테이블을 사용하는 게 무의미해진다. 
즉 클러스터가 커지지 않게 유지해야 한다.

그리고, 중복을 피해서 원래 자리에서 멀리까지 왔다면, 다시 찾거나, 수정하거나, 삭제할때도 왔던 길을 되돌아와야한다. 그리고, 내가 쏙 빠졌다면, 그 구멍을 매꿔야한다. 안그러면 나 다음으로 저장된 녀석들은 미아가 되기 때문이다. (마치 번개장터에서 실제 판매되지 않은 물건 게시물 댓글로 "판매 완료"라고 적는 것도 이와 비슷한 맥락일 수 있겠다.)
1. hashOnpenAddress

- 구조
                            [ table, size ] 

- 규칙
    1) 저장할 데이터는 중복되지 않는 key와 value가 쌍으로 이루어져있다.
    2) key를 해싱한 결과인 hashedKey를 인덱스로 사용하여 리스트에 접근하며, 충돌이 일어나지 않았다면 그 값을 저장한다.
    3) 충돌이 일어났다면 우선 linear probing 기법을 통해 빈자리를 찾는다.
    4) 빈 자리엔 [key, value]를 쌍으로 저장한다.

- 메서드 명세
    특수 메서드
    __init__
    __str__
    __getitem__ => H[key]를 통해 테이블에 접근할 수 있도록 구현 
    __setitem__ => H[key]를 통해 테이블에 접근할 수 있도록 구현

    일반 메서드
    hashFunction(key)
    findSlot(key)
    set(key, value)
    remove(key)
    search(key)

- 구현
    특수메서드
    __init__(self, size=10)
        keyTable, valueTable, size로 구성되어 있고, 각 table은 size만큼의 None으로 초기화
    __str__
        key와 value들을 요소로 갖는 리스트를 출력하도록
    __getitem__ 
        => H[key]를 통해 테이블에 접근할 수 있도록 구현 
        H[39]=TV 라고 입력하면, 실제로 39번 인덱스에 값이 저장되는 게 아니라, 
        내부적으로 해싱해서 findSlot->set 을 진행 후 return
    __setitem__ => H[key]를 통해 테이블에 접근할 수 있도록 구현


    
    일반메서드
    hashFunction(key)->(int)
    findSlot(key)->(emptyIndex)->
        해당 key를 해싱한 결과인 hashedKey를 갖고 정해진 규칙에 따라 해당 key가 들어갈 수 있는 자리를 탐색한다. 
        내 자리 => 들어갈 자리 or 들어간 자리
    set(key, value)->(key)
        findSlot을 호출하여 내 자리를 찾는다. 빈 곳이라면 삽입하고, 비어있지 않다면 수정한다.
        __getitem 구현을 위해서 등의 이유로 key값을 리턴한다.
    remove(key)->(key)
        findSolt을 호출하여 내 자리를 찾아서, 값을 지운다.
        근데 문제가 있다. 연결고리가 끊기기 때문에 이를 방지하기 위해 dummy라는 객체를 만들고 이를 key값에 저장한 뒤에 넘어간다. 
        탐색 시, key가 dummy이면 그냥 pass
    search


"""
ToDo
Quadratic probing과 Chaining 구현 => 성능 비교
연결리스트를 각 버킷에 연결하여, 충돌이 없는 구조 구현
Hash로 List를 구현
"""

</pre>